In [35]:
from keras.callbacks import ModelCheckpoint

import numpy
import math
import cv2
import time

from utility.h5data import h5DataRead

In [36]:
DNN = "srcnn"

EPOCHS = 4
BATCH_SIZE = 128

SCALE = 4
SIZE_PATCH = 32
SIZE_CONV = 7

FILENAME_TRAIN = "yayoi_dataTrain_" + str(SCALE) + "_"+ str(SIZE_PATCH) + "_"+ str(SIZE_CONV) + ".h5"
FILENAME_VALIDATE = "yayoi_dataValidate_" + str(SCALE) + "_"+ str(SIZE_PATCH) + "_"+ str(SIZE_CONV) + ".h5"
FILENAME_CHECKPOINT = "dataCheckPoint_" + DNN + "_" + str(SCALE) + ".h5"

In [37]:
# dnn configuration for srcnn
from network.srcnn import srcnn
dnn = srcnn()
dnn.layers = 4
dnn.filter = [128, 64, 64, 1]
dnn.kernelSize =  [(11,11), (3,3), (3,3), (5,5)]
dnn.in_train = True

In [38]:
tick1 = cv2.getTickCount()

In [39]:
model = dnn.network()
print(model.summary())
data, label = h5DataRead(FILENAME_TRAIN)
val_data, val_label = h5DataRead(FILENAME_VALIDATE)

checkpoint = ModelCheckpoint(FILENAME_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='min')

callbacks_list = [checkpoint]

model.fit(data, label, batch_size=BATCH_SIZE, validation_data=(val_data, val_label),
                callbacks=callbacks_list, shuffle=True, epochs=EPOCHS, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 22, 22, 128)       15616     
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 22, 22, 64)        73792     
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 22, 22, 64)        36928     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 18, 18, 1)         1601      
Total params: 127,937
Trainable params: 127,937
Non-trainable params: 0
_________________________________________________________________
None
Train on 3200 samples, validate on 750 samples
Epoch 1/4
3200/3200 [==============================] - 145s 45ms/step - loss: 0.0163 - mean_squared_error: 0.0163 - val_loss: 2.8175e-04 - val_mean_squared_error: 2.8174e-04

Epoch 00001: val_loss improved from inf t

In [40]:
score = model.evaluate(val_data, val_label, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

750/750 [==============================] - 14s 18ms/step
Test loss: 3.81764024423e-05
Test accuracy: 3.81763743571e-05


In [41]:
fileName = 'yayoi_' + DNN + "_" + str(SCALE) + "_" + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.h5'
model.save_weights(fileName)
print(fileName + ' saved')

yayoi_4_2018-07-13_092157.h5 saved


In [52]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Training processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Training processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Training processed time: " + str(tick) + " ms")

Training processed time: 49 secs 112 ms


In [53]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [54]:
tick1 = cv2.getTickCount()

IMG_NAME = "./imageTrain/yayoi_first_030.png"
IMG_OPENCV_NAME = "opencv.png"
IMG_DNN_NAME = DNN + ".png"

INTERPOLATION = cv2.INTER_CUBIC
SIZE_CONV = 7

In [55]:
dnn.in_train = False
model = dnn.network()
model.load_weights(fileName)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, None, None, 128)   15616     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, None, None, 64)    73792     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_45 (Conv2D)           (None, None, None, 1)     1601      
Total params: 127,937
Trainable params: 127,937
Non-trainable params: 0
_________________________________________________________________
None


In [56]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
shape = img.shape

Y_img = cv2.resize(img[:, :, 0], (int(shape[1] * SCALE), int(shape[0] * SCALE)), INTERPOLATION)
img = cv2.resize(img, (int(shape[1] * SCALE), int(shape[0] * SCALE)), INTERPOLATION)

#Y_img = cv2.resize(img[:, :, 0], (int(shape[1] / sizeMux), int(shape[0] / sizeMux)), cv2.INTER_CUBIC)
#Y_img = cv2.resize(Y_img, (shape[1], shape[0]), cv2.INTER_CUBIC)
#img[:, :, 0] = Y_img

img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
cv2.imwrite(IMG_OPENCV_NAME, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

True

In [57]:
Y = numpy.zeros((1, img.shape[0], img.shape[1], 1), dtype=float)
Y[0, :, :, 0] = Y_img.astype(float) / 255.

pre = model.predict(Y, batch_size=1) * 255.
pre[pre[:] > 255] = 255
pre[pre[:] < 0] = 0

pre = pre.astype(numpy.uint8)
img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

if SIZE_CONV > 0:
    img[SIZE_CONV: -SIZE_CONV, SIZE_CONV: -SIZE_CONV, 0] = pre[0, :, :, 0]
else:
    img[2:-2, 2:-2, 0] = pre[0, :, :, 0]

img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
cv2.imwrite(IMG_DNN_NAME, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

True

In [60]:
# display image
pltShow = False

if pltShow:
    im1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
    im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
    im2 = cv2.imread(OPENCV_NAME, cv2.IMREAD_COLOR)
    im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
    im3 = cv2.imread(SRCNN_NAME, cv2.IMREAD_COLOR)
    im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)
    
    plt.figure(num='comparison',figsize=(16,16))

    plt.subplot(1,3,1)
    plt.title('origin image')
    plt.imshow(im1)

    plt.subplot(1,3,2)
    plt.title('OpenCV')
    plt.imshow(im2)

    plt.subplot(1,3,3)
    plt.title(DNN)
    plt.imshow(im3)

In [61]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Inference processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Inference processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Inference processed time: " + str(tick) + " ms")

processed time: 3 mins 56 secs 521 ms
